In [1]:
from pyspark import SparkConf, SparkContext
conf=SparkConf().setMaster("local").setAppName("spark_sql_basic")
sc=SparkContext(conf=conf)

# RDD만을 이용한 데이터 추출

In [8]:
movies_rdd=sc.parallelize([
    (1, ("어벤져스", "마블")),(2, ("슈퍼맨", "디씨")),  (3, ("배트맨", "디씨"))
    ,  (4, ("겨울왕국", "디지니")),  (5, ("아이언맨", "마블"))
])
attendances_rdd = sc.parallelize([
    (1,(13433958,"KR")),
    (2,(1534958,"KR")),
    (3,(13894958,"KR")),
    (4,(13158,"KR")),
    (5,(12958,"KR")),

])

마블 영화 중 관객 수가 500만 이상인 영화 가져오기
1) inner join -> filter by movie -> filter by attendacne
2) filter by movie, filter by attendance -> inner join

In [9]:
#case 1 join 먼저  filter 나중
movie_att = movies_rdd.join(attendances_rdd)
movie_att.filter(
    lambda x: x[1][0][1] == "마블"and x[1][1][0]>5000000
    ).collect()

[(1, (('어벤져스', '마블'), (13433958, 'KR')))]

In [14]:
#case 2, filter 먼저 join 나중에
filtered_movies= movies_rdd.filter(lambda x : x[1][1]=="마블")
filtered_att = attendances_rdd.filter(lambda x : x[1][0] > 5000000)

filtered_movies.join(filtered_att).collect()

[(1, (('어벤져스', '마블'), (13433958, 'KR')))]

동일한 결과지만 필터를 먼저 수행해서 가져올 데이터를 걸러낸 다음 조인하는 case2가 훨씬 효율적이다

# Spark SQL 사용해보기

1. SparkSession  만들기
    Sparkcontext에 해당하며, 새로운 스파크 어플리케이션을 생성

In [21]:
from pyspark.sql import SparkSession
spark = SparkSession .builder.master("local").appName("spark-sql").getOrCreate()

In [26]:
movies=([
    (1, "어벤져스", "마블",2012),(2, "슈퍼맨", "디씨",2013),  (3, "배트맨", "디씨",2008)
    ,  (4, "겨울왕국", "디지니",2014),  (5,"아이언맨", "마블",2008)
])

In [27]:
movie_schema=["id","name","company","year"]

데이터 프레임 확인하기

In [28]:
df=spark.createDataFrame(data=movies, schema=movie_schema)

In [29]:
# 스키마 타입 확인
df.dtypes

[('id', 'bigint'),
 ('name', 'string'),
 ('company', 'string'),
 ('year', 'bigint')]

In [30]:
df.show()

+---+--------+-------+----+
| id|    name|company|year|
+---+--------+-------+----+
|  1|어벤져스|   마블|2012|
|  2|  슈퍼맨|   디씨|2013|
|  3|  배트맨|   디씨|2008|
|  4|겨울왕국| 디지니|2014|
|  5|아이언맨|   마블|2008|
+---+--------+-------+----+



In [31]:
df.createOrReplaceTempView("movies")

In [32]:
query="""

    SELECT name
    FROM movies

"""
spark.sql(query).show()


+--------+
|    name|
+--------+
|어벤져스|
|  슈퍼맨|
|  배트맨|
|겨울왕국|
|아이언맨|
+--------+



In [33]:
query="""

    SELECT *
    FROM movies

"""
spark.sql(query).show()

+---+--------+-------+----+
| id|    name|company|year|
+---+--------+-------+----+
|  1|어벤져스|   마블|2012|
|  2|  슈퍼맨|   디씨|2013|
|  3|  배트맨|   디씨|2008|
|  4|겨울왕국| 디지니|2014|
|  5|아이언맨|   마블|2008|
+---+--------+-------+----+



In [ ]:
#2012 년 이전에 개봉한 영화의 이름과 회사를 출력

In [34]:
query="""

 SELECT name,  company
 FROM movies
 where year<2012
 
"""
spark.sql(query).show()

+--------+-------+
|    name|company|
+--------+-------+
|  배트맨|   디씨|
|아이언맨|   마블|
+--------+-------+



In [36]:
#like 문자열 데이터에서 특정 단어나 문장을 포함한 데이터를 찾을 때 사용
# % 기호를 사용해서 문장이 매칭되는지 확인 가능

#제목이 ~맨으로 끝나는 모든 데이터 출력

query="""

 SELECT *
 FROM movies
 where name LIKE '%맨'
 
"""
spark.sql(query).show()

+---+--------+-------+----+
| id|    name|company|year|
+---+--------+-------+----+
|  2|  슈퍼맨|   디씨|2013|
|  3|  배트맨|   디씨|2008|
|  5|아이언맨|   마블|2008|
+---+--------+-------+----+



In [37]:
#'이'가 들어간 영화찾기

query="""

 SELECT *
 FROM movies
 where name LIKE '%이%'
 
"""
spark.sql(query).show()

+---+--------+-------+----+
| id|    name|company|year|
+---+--------+-------+----+
|  5|아이언맨|   마블|2008|
+---+--------+-------+----+



In [40]:
# BETWEEN 특정 데이터와 데이터 사이를 조회

#개봉월이 4월~8월 사이 출력


query="""

     -SELECT *
 FROM movies
 where YEAR BETWEEN 2010 AND 2018
 
"""
spark.sql(query).show()

+---+--------+-------+----+
| id|    name|company|year|
+---+--------+-------+----+
|  1|어벤져스|   마블|2012|
|  2|  슈퍼맨|   디씨|2013|
|  4|겨울왕국| 디지니|2014|
+---+--------+-------+----+



In [43]:
#AND연산

query="""

SELECT *
 FROM movies
 where name LIKE"%맨" AND year >= 2012
 
"""
spark.sql(query).show()

+---+------+-------+----+
| id|  name|company|year|
+---+------+-------+----+
|  2|슈퍼맨|   디씨|2013|
+---+------+-------+----+



In [46]:
# in 연산 활용
# 컬럼명 in(값1, 값2,,,)

query="""

SELECT *
 FROM movies
 where company in ('마블','디씨')
 
"""
spark.sql(query).show()

+---+--------+-------+----+
| id|    name|company|year|
+---+--------+-------+----+
|  1|어벤져스|   마블|2012|
|  2|  슈퍼맨|   디씨|2013|
|  3|  배트맨|   디씨|2008|
|  5|아이언맨|   마블|2008|
+---+--------+-------+----+



In [47]:
#회사가 '마'로 시작을 하거나 , '즈'로 끝나는 영화

query="""

SELECT *
 FROM movies
 where company LIKE '마%' OR company LIKE '%즈'
 
"""
spark.sql(query).show()

+---+--------+-------+----+
| id|    name|company|year|
+---+--------+-------+----+
|  1|어벤져스|   마블|2012|
|  5|아이언맨|   마블|2008|
+---+--------+-------+----+



'ORDER BY'절 : 정렬

- ASC: 오르차순
- DESC: 내림차순


In [49]:
query="""

SELECT *
 FROM movies
 ORDER BY year desc
 
"""
spark.sql(query).show()

+---+--------+-------+----+
| id|    name|company|year|
+---+--------+-------+----+
|  4|겨울왕국| 디지니|2014|
|  2|  슈퍼맨|   디씨|2013|
|  1|어벤져스|   마블|2012|
|  3|  배트맨|   디씨|2008|
|  5|아이언맨|   마블|2008|
+---+--------+-------+----+



In [50]:
query="""

SELECT *
 FROM movies
 ORDER BY year asc
 
"""
spark.sql(query).show()

+---+--------+-------+----+
| id|    name|company|year|
+---+--------+-------+----+
|  3|  배트맨|   디씨|2008|
|  5|아이언맨|   마블|2008|
|  1|어벤져스|   마블|2012|
|  2|  슈퍼맨|   디씨|2013|
|  4|겨울왕국| 디지니|2014|
+---+--------+-------+----+



- count : 개수 세기
- mean : 평균 구하기
- sum : 총합

In [53]:
query = """
SELECT count(*) as movie_count
    FROM movies

"""
spark.sql(query).show()

+-----------+
|movie_count|
+-----------+
|          5|
+-----------+



In [55]:
query = """
SELECT count(*) as movie_count
    FROM movies
    WHERE company = "디씨"

"""
spark.sql(query).show()

+-----------+
|movie_count|
+-----------+
|          2|
+-----------+



# Join 구현하기

In [72]:
attendances=[
    (1,13433958.,"KR"),
    (2,1534958.,"KR"),
    (3,13894958.,"KR"),
    (4,13158.,"KR"),
    (5,12958.,"KR")

]

직접 스키마 지정해보기

In [73]:
from pyspark.sql.types import StringType, FloatType, IntegerType, StructType, StructField

In [74]:
att_schema = StructType([  # 모든 컬럼의 타입을 통칭 - 컬럼 데이터의 집합
    StructField("id",IntegerType(),True), #StructField 컬럼
    StructField("att",FloatType(),True),
    StructField("theater_country",StringType(),True),
    
])

In [75]:
att_df = spark.createDataFrame(
    data=attendances,
    schema=att_schema
)

att_df.dtypes

[('id', 'int'), ('att', 'float'), ('theater_country', 'string')]

In [76]:
att_df.createOrReplaceTempView("att")

In [77]:
#쿼리를 사용하지 않고 모든 데이터를 확인
#DataFrame API를 사용
att_df.select("*").show()

+---+-----------+---------------+
| id|        att|theater_country|
+---+-----------+---------------+
|  1|1.3433958E7|             KR|
|  2|  1534958.0|             KR|
|  3|1.3894958E7|             KR|
|  4|    13158.0|             KR|
|  5|    12958.0|             KR|
+---+-----------+---------------+



In [84]:
query = """
SELECT movies.id, movies.name, movies.company, att.att
    FROM movies
    JOIN att ON movies.id=att.id

"""

spark.sql(query).show()

+---+--------+-------+-----------+
| id|    name|company|        att|
+---+--------+-------+-----------+
|  1|어벤져스|   마블|1.3433958E7|
|  2|  슈퍼맨|   디씨|  1534958.0|
|  3|  배트맨|   디씨|1.3894958E7|
|  4|겨울왕국| 디지니|    13158.0|
|  5|아이언맨|   마블|    12958.0|
+---+--------+-------+-----------+



# 데이터 프레임 API

In [87]:
#select
#data frame api의 결과는 Transformations 작업이기 때문에 collect, show를 통해서 데이터를 확인
df.select("*").collect()

[Row(id=1, name='어벤져스', company='마블', year=2012),
 Row(id=2, name='슈퍼맨', company='디씨', year=2013),
 Row(id=3, name='배트맨', company='디씨', year=2008),
 Row(id=4, name='겨울왕국', company='디지니', year=2014),
 Row(id=5, name='아이언맨', company='마블', year=2008)]

In [88]:
df.select("name","company").collect()

[Row(name='어벤져스', company='마블'),
 Row(name='슈퍼맨', company='디씨'),
 Row(name='배트맨', company='디씨'),
 Row(name='겨울왕국', company='디지니'),
 Row(name='아이언맨', company='마블')]

In [90]:
df.select(df.name,(df.year-2000).alias("year")).show()

+--------+----+
|    name|year|
+--------+----+
|어벤져스|  12|
|  슈퍼맨|  13|
|  배트맨|   8|
|겨울왕국|  14|
|아이언맨|   8|
+--------+----+



In [92]:
#agg:Aggregate 의 약자로써, 그룹핑 후 데이터를 하나로 합쳐주는 역할

df.agg({"id": "count"}).collect()

[Row(count(id)=5)]

In [95]:
from pyspark.sql import functions as F
df.agg(F.min(df.year)).collect()

[Row(min(year)=2008)]

In [96]:
query = """
SELECT count(*) as movie_count
    FROM movies
    WHERE company="DC"


"""

spark.sql(query).collect()

[Row(movie_count=0)]

In [97]:
df.groupBy().avg().collect()

[Row(avg(id)=3.0, avg(year)=2011.0)]

In [101]:
#select, where, orderBy 절 사용
marvel_df = df.select("name","company","year").where("company=='마블'").orderBy("id")
marvel_df.collect() 

[Row(name='어벤져스', company='마블', year=2012),
 Row(name='아이언맨', company='마블', year=2008)]

In [102]:
sc.stop()